In [2]:
import sys
!{sys.executable} -m pip install pyarrow

import pandas as pd

db = pd.read_parquet(r"C:\KDT\bok_pjt\team2_forecast_pjt\preprocessing\df_final_preprocessed.parquet")

In [ ]:
from collections import Counter
from itertools import chain

token_counts = Counter(chain.from_iterable(db['tokens']))

print(f"전체 유니크 토큰 수: {len(token_counts):,}개")

전체 유니크 토큰 수: 9,153,263개


In [ ]:
keep_tokens = {token for token, count in token_counts.items() if count >= 15}

print(f"필터링 후 남은 토큰 수: {len(keep_tokens):,}개")
print(f"제거된 희소 토큰 수: {len(token_counts) - len(keep_tokens):,}개")

필터링 후 남은 토큰 수: 27,693개
제거된 희소 토큰 수: 9,125,570개


In [ ]:
from tqdm import tqdm

tqdm.pandas()

def filter_rare_tokens(token_list):
    return [t for t in token_list if t in keep_tokens]

print("✨ 희소 단어 제거 작업을 시작합니다...")
df_del = db.copy()
df_del['tokens'] = db['tokens'].progress_apply(filter_rare_tokens)

df_del = df_del[db['tokens'].map(len) > 0].reset_index(drop=True)
print(f"최종 남은 데이터 개수: {len(df_del):,}건")

✨ 희소 단어 제거 작업을 시작합니다...


100%|██████████| 4974737/4974737 [00:06<00:00, 715882.05it/s]


최종 남은 데이터 개수: 4,953,607건


In [8]:
df_del.head(10)

,doc_id,date,content,tokens,category,source
0,0,2025-11-14,14일 서울 중구 하나은행 본점 딜링룸 현황판에 코스피가 표시돼 있다.,"[중구/NNG;본점/NNG;딜링룸/NNG;현황/NNG;판/NNG, 코스피/NNG;표...",뉴스,매일경제
1,0,2025-11-14,나흘 연속 상승했던 코스피가 3 대 급락하며 4000선을 간신히 지켰다.,[],뉴스,매일경제
2,0,2025-11-14,간밤 미국 셧다운 해제에도 불구하고 금리인하 기대감이 약화한 데다 고평가 논란이 불...,[풀이/NNG;되/VV],뉴스,매일경제
3,0,2025-11-14,14일 코스피는 전일 대비 159.06포인트 3.81 내린 4011.57에 거래를 ...,"[코스피/NNG;대비/NNG;포인트/NNG;내리/VV;거래/NNG, 마치/VV]",뉴스,매일경제
4,0,2025-11-14,이날 지수는 전일 대비 108.72포인트 2.61 급락한 4061.91에 출발한 뒤...,[],뉴스,매일경제
5,0,2025-11-14,간밤 뉴욕증시가 인공지능 AI 산업 고평가 우려와 미 연방준비제도 Fed 연준 인사...,[],뉴스,매일경제
6,0,2025-11-14,13일 미국 동부시간 뉴욕증권거래소 NYSE 에서 스탠더드앤드푸어스 S P 500 ...,[포인트/NNG;내리/VV;장/NNG;마감/NNG;하/VV],뉴스,매일경제
7,0,2025-11-14,다우존스30산업평균지수는 전일 대비 797.60포인트 1.65 하락한 4만7457....,"[다우존스산업평균지수/NNG;대비/NNG;포인트/NNG;하락/NNG;거래/NNG, ...",뉴스,매일경제
8,0,2025-11-14,또 엔비디아 3.56 AMD 4.21 팔란티어 6.53 등 AI 관련주가 급락했고 ...,[하락/NNG;하/VV],뉴스,매일경제
9,0,2025-11-14,이경민 대신증권 연구원은 금리 인하 기대감이 후퇴하면서 성장 기술주 섹터의 부담이 ...,[],뉴스,매일경제


In [9]:
output_file = 'df_filtered_15.parquet'
df_del.to_parquet(output_file, index=False)
print(f"✅ 깨끗하게 정돈된 마스터 파일이 '{output_file}'로 저장되었습니다!")

✅ 깨끗하게 정돈된 마스터 파일이 'df_filtered_15.parquet'로 저장되었습니다!
